In [1]:
import torch
import scvi
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt

sc.set_figure_params(dpi=100, frameon=False, color_map='Reds', facecolor=None)
sc.logging.print_header()

Global seed set to 0
/home/yuge/miniconda3/envs/fa_base/lib/python3.8/site-packages/numba/core/cpu.py:78: UserWarning: Numba extension module 'sparse._numba_extension' failed to load due to 'ModuleNotFoundError(No module named 'numba.targets')'.
  numba.core.entrypoints.init_all()
2023-01-29 15:55:56.229494: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-29 15:55:56.229559: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-29 15:55:56.229563: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing librari

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.3 scipy==1.10.0 pandas==1.5.0 scikit-learn==1.1.2 statsmodels==0.13.2 python-igraph==0.10.3 pynndescent==0.5.7


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
hlca_path = '../datasets/hlca.h5ad'
adata = ad.read(hlca_path)
adata

AnnData object with n_obs × n_vars = 584944 × 2000
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_ne

In [4]:
model = scvi.model.SCANVI.load('../models/scanvi_model/', adata)
model

INFO     File ../models/scanvi_model/model.pt already downloaded                             


/home/yuge/miniconda3/envs/fa_base/lib/python3.8/site-packages/scvi/model/base/_utils.py:134: UserWarning: var_names for adata passed in does not match var_names of adata used to train the model. For valid results, the vars need to be the same and in the same order as the adata used to train the model.
  warnings.warn(
/home/yuge/miniconda3/envs/fa_base/lib/python3.8/site-packages/scvi/data/fields/_layer_field.py:78: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(
/home/yuge/miniconda3/envs/fa_base/lib/python3.8/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(
/home/yuge/miniconda3/envs/fa_base/lib/python3.8/site-packages/scvi/data/_utils.py:156: UserWarning: Category 18 in adata.obs['_scvi_labels'] has fewer than 3 cells. Models may not train properly.
  warnings.warn(


ScanVI Model with the following params: 
unlabeled_category: unlabeled, n_hidden: 128, n_latent: 30, n_layers: 2, dropout_rate: 0.1, 
dispersion: gene, gene_likelihood: nb
Training status: Trained

In [5]:
preds = model.predict(adata)

In [6]:
labels = adata.obs.scanvi_label.values

In [7]:
from sklearn.metrics import classification_report
print(classification_report(labels, preds))

/home/yuge/miniconda3/envs/fa_base/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yuge/miniconda3/envs/fa_base/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                         precision    recall  f1-score   support

                    AT1       0.00      0.00      0.00      9138
                    AT2       0.87      0.97      0.92     58398
            Arterial EC       0.00      0.00      0.00      6031
         B cell lineage       0.00      0.00      0.00      6361
                  Basal       0.89      0.95      0.92     93000
     Bronchial Vessel 1       0.00      0.00      0.00       455
     Bronchial Vessel 2       0.00      0.00      0.00       235
              Capillary       0.42      0.99      0.59     20597
               Ciliated       0.94      0.98      0.96     42295
        Dendritic cells       0.00      0.00      0.00      7813
     Fibroblast lineage       0.78      0.99      0.87     20549
KRT5- KRT17+ epithelial       0.00      0.00      0.00        11
           Lymphatic EC       0.00      0.00      0.00      4622
            Macrophages       0.73      1.00      0.84    106551
             Mast cells 

/home/yuge/miniconda3/envs/fa_base/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### kNN mapping

In [8]:
def perform_label_transfer(ref_emb, query_emb, cell_type_column, k=15):
    # calculate an object representing the joing neighbor graph of ref + query
    ing = sc.tl.Ingest(ref_emb)
    ing.fit(query_emb)
    ing.neighbors(k=k)
    # calculate distances to top k neighbors for each cell and store indices
    # of neighbor cells
    top_k_distances, top_k_indices = (
        ing._distances,
        ing._indices,
    )
    # transform distances with Gaussian kernel (?)
    stds = np.std(top_k_distances, axis=1)
    stds = (2.0 / stds) ** 2  # don't know why the first 2.0
    stds = stds.reshape(-1, 1)
    top_k_distances_tilda = np.exp(-np.true_divide(top_k_distances, stds))
    # normalize so that transformed distances sum to 1
    weights = top_k_distances_tilda / np.sum(
        top_k_distances_tilda, axis=1, keepdims=True
    )
    # initialize empty series to store predicted labels and matching
    # uncertaintites for every query cell
    uncertainties = pd.Series(index=query_emb.obs_names, dtype="float64")
    pred_labels = pd.Series(index=query_emb.obs_names, dtype="object")
    # now loop through query cells
    y_train_labels = ref_emb.obs[cell_type_column].values
    for i in range(len(weights)):
        # store cell types present among neighbors in reference
        unique_labels = np.unique(y_train_labels[top_k_indices[i]])
        # store best label and matching probability so far
        best_label, best_prob = None, 0.0
        # now loop through all cell types present among the cell's neighbors:
        for candidate_label in unique_labels:
            candidate_prob = weights[
                i, y_train_labels[top_k_indices[i]] == candidate_label
            ].sum()
            if best_prob < candidate_prob:
                best_prob = candidate_prob
                best_label = candidate_label
        else:
            pred_label = best_label
        # store best label and matching uncertainty
        uncertainties.iloc[i] = max(1 - best_prob, 0)
        pred_labels.iloc[i] = pred_label
    # print info
    print(
        "Storing transferred labels in your query adata under .obs column:",
        f"transf_{cell_type_column}",
    )
    print(
        "Storing label transfer uncertainties in your query adata under .obs column:",
        f"transf_{cell_type_column}_unc",
    )
    # store results
    query_emb.obs[f"transf_{cell_type_column}"] = pred_labels
    query_emb.obs[f"transf_{cell_type_column}_unc"] = uncertainties

In [22]:
ref_emb = ad.AnnData(adata.obsm['X_scanvi_emb'], obs=adata.obs)
sc.pp.neighbors(ref_emb)
# ref_emb.uns['neighbors'] = adata.uns['neighbors']
# ref_emb.obsm['X_umap'] = adata.obsm['X_umap']
# ref_emb.obsp['connectivities'] = adata.obsp['connectivities']
# ref_emb.obsp['distances'] = adata.obsp['distances']
ref_emb

AnnData object with n_obs × n_vars = 584944 × 30
    obs: 'sample', 'original_celltype_ann', 'study_long', 'study', 'last_author_PI', 'subject_ID', 'subject_ID_as_published', 'pre_or_postnatal', 'age_in_years', 'age_range', 'sex', 'ethnicity', 'mixed_ethnicity', 'smoking_status', 'smoking_history', 'BMI', 'known_lung_disease', 'condition', 'subject_type', 'cause_of_death', 'sample_type', 'anatomical_region_coarse', 'anatomical_region_detailed', 'tissue_dissociation_protocol', 'cells_or_nuclei', 'single_cell_platform', "3'_or_5'", 'enrichment', 'sequencing_platform', 'reference_genome_coarse', 'ensembl_release_reference_genome', 'cell_ranger_version', 'disease_status', 'fresh_or_frozen', 'cultured', 'cell_viability_%', 'comments', 'Processing_site', 'dataset', 'anatomical_region_level_1', 'anatomical_region_level_2', 'anatomical_region_level_3', 'anatomical_region_highest_res', 'age', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'ann_highest_res', 'ann_new'

In [ ]:
perform_label_transfer(
    ref_emb=ref_emb, query_emb=ref_emb, cell_type_column="scanvi_label"
)

## From Jan:

In [4]:
from uncertainty_atlas.models.mimo import Mimo

In [6]:
model = Mimo.load_from_checkpoint(
    "/home/yuge/feature-attribution-sc/models/scanvi_model/model_params.pt"
)

KeyError: 'pytorch-lightning_version'

In [ ]:
# hlca = sc.read(cfg.data_file)
# hlca_core = hlca[hlca.obs.core_or_extension == "core"]
# cfg.dataloader
# {'batch_size': 1024, 'num_workers': 0, 'label_col': '${label_col}'}
# meyer_index = hlca[hlca.obs.study == "Meyer_2021"].obs.index
dataloader = get_dataloader(
    hlca,
    hlca.obs.index,
    shuffle=False,
    {'batch_size': 1024, 'num_workers': 0, 'label_col': 'manual_ann'})

In [ ]:
trainer = Trainer()

In [ ]:
predictions = trainer.predict(model, dataloader)

In [ ]:
preds = torch.cat(predictions).numpy()
# preds.shape
# (129340, 10, 61)
y_proba = preds.mean(1)
y_pred = meyer_y_proba.argmax(-1)
hlca.obs["mimo_preds"] = pd.Categorical(
    pd.Series(y_pred).map(
        dict(enumerate(hlca.obs.manual_ann.cat.categories))
    )
)